Random forest survival on original data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, inspect, sys
from sksurv.ensemble import RandomSurvivalForest
from sksurv.datasets import load_gbsg2
import eli5
from eli5.sklearn import PermutationImportance

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

This NB does random survival forest analysis on the real data. 

In [ ]:
data = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/Survival_df.csv'),
                  index_col=0)


In [ ]:
data_x = data.iloc[:, 2:]
data_y = data.iloc[:, 0:2]
data_y_num = data_y.to_records(index=False)


## Random survival forest

In [ ]:
rsf = RandomSurvivalForest(n_estimators=1000,
                           max_features="auto",
                          oob_score=True,
                          )
rsf.fit(data_x, data_y_num)


## Out of bag score (Concordance index)

This is a measure of how well the model orders the sample. The model was trained on all the data because there is so little. This performs worse than random.

In [ ]:
rsf.oob_score_

## Feature importance by permutation

This estimates the importance of each feature by permutating it and looking at the effect on the model. This is not good when the features are correlated as they are in this case.

In [ ]:
perm = PermutationImportance(rsf, n_iter=15)
perm.fit(data_x, data_y_num)

In [ ]:
feature_names = data_x.columns.tolist()
eli5.show_weights(perm, feature_names=feature_names)